In [1]:
# If running on Colab, uncomment the next lines
# %tensorflow_version 2.x
# device_name = tf.test.gpu_device_name()
# print('GPU not found.') if device_name != '/device:GPU:0' else print('Found GPU at: {}'.format(device_name))

In [2]:
import tensorflow as tf
from utils.dataset_functions import create_dataset, unwrap_client_data
from utils.image_processing import *
from utils.unet import initialize_unet
from utils.federated_averaging import federated_averaging
from utils.visualization import visualize_results_testset

In [3]:
imagePath0 = 'merged_data/0/' # Post Spreading Images
imagePath1 = 'merged_data/1/' # Post Fusion Images
npyPath = 'merged_data/annotations/' # Annotations

clientIdentifierDict = {
    'client1' : ['000001', '000002', '000003'],
    'client2' : ['000004', '000005', '000006'],
    'client3' : ['000007', '000008', '000009'],
    'client4' : ['000010', '000011', '000012'],
    'client5' : ['000013', '000014', '000015'],
    'client6' : ['000016', '000017', '000018'],
    'client7' : ['000019', '000020', '000021'],
    'client8' : ['000022', '000023', '000024']
}

datasetImageDict, datasetMaskDict = create_dataset(clientIdentifierDict, 
                                             imagePath0, imagePath1, npyPath, 
                                             tileSize = 128)


client1...
  처리 중: 3/3 파일 완료 (현재 타일 수: 75)
Contains 3 images...
Tiled Image Tensor Shape:  (75, 128, 128, 2)
Tiled Mask Shape:  (75, 128, 128)

client2...
  처리 중: 3/3 파일 완료 (현재 타일 수: 75)
Contains 3 images...
Tiled Image Tensor Shape:  (75, 128, 128, 2)
Tiled Mask Shape:  (75, 128, 128)

client3...
  처리 중: 3/3 파일 완료 (현재 타일 수: 75)
Contains 3 images...
Tiled Image Tensor Shape:  (75, 128, 128, 2)
Tiled Mask Shape:  (75, 128, 128)

client4...
  처리 중: 3/3 파일 완료 (현재 타일 수: 75)
Contains 3 images...
Tiled Image Tensor Shape:  (75, 128, 128, 2)
Tiled Mask Shape:  (75, 128, 128)

client5...
  처리 중: 3/3 파일 완료 (현재 타일 수: 75)
Contains 3 images...
Tiled Image Tensor Shape:  (75, 128, 128, 2)
Tiled Mask Shape:  (75, 128, 128)

client6...
  처리 중: 3/3 파일 완료 (현재 타일 수: 75)
Contains 3 images...
Tiled Image Tensor Shape:  (75, 128, 128, 2)
Tiled Mask Shape:  (75, 128, 128)

client7...
  처리 중: 3/3 파일 완료 (현재 타일 수: 75)
Contains 3 images...
Tiled Image Tensor Shape:  (75, 128, 128, 2)
Tiled Mask Shape:  (75, 12

In [4]:
trainClients = ['client1', 'client2', 'client3', 'client4', 
                'client5', 'client7', 'client8']
testClients = ['client6']

clientIDs, NCLIENTS = trainClients, len(trainClients)

Client Data is stored in two separate dictionaries, one for tiled image tensors (X) and other for segmentation masks (Y). Both dictionaries are keyed by `clientID`.

Training Dictionaries - `imageDict`(X) and `segMaskDict`(Y)

Testing Dictionaries - `imageDictTest`(X) and `segMaskDictTest`(Y)

In [5]:
# Train Data
imageDict, segMaskDict = {}, {}
for clientID in trainClients:
    imageDict[clientID] = datasetImageDict[clientID]
    segMaskDict[clientID] = datasetMaskDict[clientID]

# Test Data
imageDictTest, segMaskDictTest = {}, {}
for clientID in testClients:
    imageDictTest[clientID] = datasetImageDict[clientID]
    segMaskDictTest[clientID] = datasetMaskDict[clientID]
# unwrap test data from dictionary for easier model evaluation
testImages, testMasks = unwrap_client_data(imageDictTest, segMaskDictTest, testClients)

In [6]:
# Set hyperparameters for FL
# For demonstration, we use only 2 server rounds with 5 local epochs
SERVER_ROUNDS, LOCAL_EPOCHS, LOCAL_BATCH_SIZE, LOCAL_LEARNING_RATE = 2, 5, 32, 8e-05

# Initialize the global model
model = initialize_unet()

Model: "u-net"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │        608 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 64,    │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 64, 64,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 32, 32,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 32, 32,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 16, 16,    │          0 │ conv2d_2[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 16, 16,    │    295,168 │ max_pooling2d_2[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 8, 8, 256) │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 8, 8, 512) │  1,180,160 │ max_pooling2d_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 4, 4, 512) │          0 │ conv2d_4[0][0]    │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d       │ (None, 8, 8, 512) │          0 │ max_pooling2d_4[… │
│ (UpSampling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 8, 8, 512) │    262,656 │ up_sampling2d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 8, 8,      │          0 │ conv2d_5[0][0],   │
│ (Concatenate)       │ 1024)             │            │ conv2d_4[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 8, 8, 512) │  4,719,104 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_1     │ (None, 16, 16,    │          0 │ conv2d_6[0][0]    │
│ (UpSampling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 16, 16,    │    131,328 │ up_sampling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 16, 16,    │          0 │ conv2d_7[0][0], 

 Total params: 8,291,907 (31.63 MB)

 Trainable params: 8,291,907 (31.63 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Run (simulate) FL
# Please use GPU/Colab to avoid very long training time
# https://colab.research.google.com/notebooks/intro.ipynb?utm_source=scs-index
# https://colab.research.google.com/notebooks/gpu.ipynb

model, serverWeights, lossDict, testLoss, accuracyDict, testAccuracy = federated_averaging(model,
                        SERVER_ROUNDS, LOCAL_EPOCHS, LOCAL_BATCH_SIZE,
                        LOCAL_LEARNING_RATE,
                        clientIDs, imageDict, segMaskDict,
                        testImages, testMasks)

------ Server Epoch 0 ------
Running local updates for client1...
Epoch 1/5
1/3 ━━━━━━━━━━━━━━━━━━━━ 8s 4s/step - accuracy: 0.2375 - loss: 1.1250

KeyboardInterrupt: 

In [8]:
# Visualize results

# Since a well-trained model is available, we use it for demonstration.
fl_model = tf.keras.models.load_model('saved_models/FL_30_10_32_8e05_HoldoutPart0708.h5')

import json
import h5py

# HDF5 파일에서 모델 구조를 읽고 reduction='auto'를 수정
def fix_model_config(filepath):
    """모델 파일에서 reduction='auto'를 'sum_over_batch_size'로 변경"""
    with h5py.File(filepath, 'r+') as f:
        if 'model_weights' in f:
            # 모델 설정 수정
            if 'model_config' in f.attrs:
                config_str = f.attrs['model_config']
                if isinstance(config_str, bytes):
                    config_str = config_str.decode('utf-8')
                config = json.loads(config_str)
                
                # 재귀적으로 reduction='auto' 찾아서 수정
                def fix_reduction(obj):
                    if isinstance(obj, dict):
                        if 'reduction' in obj and obj['reduction'] == 'auto':
                            obj['reduction'] = 'sum_over_batch_size'
                        for key, value in obj.items():
                            fix_reduction(value)
                    elif isinstance(obj, list):
                        for item in obj:
                            fix_reduction(item)
                
                fix_reduction(config)
                f.attrs['model_config'] = json.dumps(config).encode('utf-8')

# 모델 파일 수정 시도 (실패해도 계속 진행)
try:
    fix_model_config('saved_models/FL_30_10_32_8e05_HoldoutPart0708.h5')
except:
    pass

# custom_objects를 사용하여 호환성 문제 해결
def create_loss_with_fixed_reduction(loss_class):
    """reduction='auto'를 'sum_over_batch_size'로 변환하는 손실 함수 래퍼"""
    def wrapper(*args, **kwargs):
        if 'reduction' in kwargs and kwargs['reduction'] == 'auto':
            kwargs['reduction'] = 'sum_over_batch_size'
        return loss_class(*args, **kwargs)
    return wrapper

# custom_objects 생성
custom_objects = {
    'SparseCategoricalCrossentropy': create_loss_with_fixed_reduction(
        tf.keras.losses.SparseCategoricalCrossentropy
    ),
    'CategoricalCrossentropy': create_loss_with_fixed_reduction(
        tf.keras.losses.CategoricalCrossentropy
    ),
    'BinaryCrossentropy': create_loss_with_fixed_reduction(
        tf.keras.losses.BinaryCrossentropy
    ),
}

# 모델 로드 (compile=False로 로드하여 호환성 문제 우회)
try:
    fl_model = tf.keras.models.load_model(
        'saved_models/FL_30_10_32_8e05_HoldoutPart0708.h5',
        compile=False,
        custom_objects=custom_objects
    )
except Exception as e:
    # compile=False로도 실패하면, 모델 가중치만 로드
    print(f"모델 로드 중 오류 발생: {e}")
    print("모델 가중치만 로드합니다...")
    # 새 모델 생성 후 가중치만 로드
    fl_model = initialize_unet()
    fl_model.load_weights('saved_models/FL_30_10_32_8e05_HoldoutPart0708.h5')

visualize_results_testset(fl_model, imageDictTest, segMaskDictTest,
                              testClients, clientIdentifierDict)

d:\iot\FLAM\venv311\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


ValueError: Invalid value for argument `reduction`. Expected one of {'mean_with_sample_weight', 'sum', 'sum_over_batch_size', 'none', None, 'mean'}. Received: reduction=auto